In [3]:
import os, types
import numpy as np
import pandas as pd
def __iter__(self): return 0
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
#import ImputingValues as t
%matplotlib inline
data_path = 'C:/Users/ruchi/OneDrive/projects/data/worldBank_waste/city_level_data_0_0.csv'
df_city = pd.read_csv(data_path )
df_city.describe(include='all')
#df_city = df_city.dropna(thresh=3, axis=1)
columns_name_indices = ['iso3c', 'region_id', 'country_name', 'income_id', 'city_name', 'total_msw_total_msw_generated_tons_year', 'population_population_number_of_people']
predict_columns_name_indices = ['iso3c', 'region_id', 'country_name', 'income_id', 'city_name', 'population_population_number_of_people']
#choose only those cities whose msw generation is reported
df_city = df_city[df_city['total_msw_total_msw_generated_tons_year'].notna()]
df_city['total_msw_total_msw_generated_tons_year'] = pd.to_numeric(df_city['total_msw_total_msw_generated_tons_year'], errors='coerce')
df_city = df_city.set_index(columns_name_indices)
df_city.describe(include='all')

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
    
#print(input_df.head)#['population_population_number_of_people'].value_counts(bins = 4))#.plot.bar()#
    
#grouped = input_df.groupby(levels)#.value_counts#filter(like='composition').plot.bar(stacked=True, rot=75).legend(bbox_to_anchor=(1.04,1), loc="upper left") 
#grouped_occurences = grouped.size().to_frame('occurences')#.reset_index()
#grouped_occurences.unstack().plot(kind='bar', stacked=True)#.plot.bar(stacked=True)
#print(grouped_occurences)
#plt.xticks(rotation=90)
#plt.show()
    
#for name, group in grouped:        
#print("NAME",name)
#print(group)#grouped.get_group(name).index.get_level_values(levels[2]).value_counts(bins = 4).plot.bar()#axs[row])
#plt.show()
#plt.xticks(rotation=90)
#plt.show()

grouped = df_city.groupby('income_id')#.value_counts#filter(like='composition').plot.bar(stacked=True, rot=75).legend(bbox_to_anchor=(1.04,1), loc="upper left") 
for name, group in grouped:
    print(name)

In [ ]:
#group column data by the information it provides and name the groups 

informal_impact_df = df_city.filter(like='informal').dropna(axis=0,how='all')
informal_impact_df.name = 'informal_impact_df'
composition_df = df_city.filter(like='composition').dropna(axis=0,how='all')
composition_df.name = 'composition_df'
separation_df = df_city.filter(like='separation').dropna(axis=0,how='all')
separation_df.name = 'separation_df'
transportation_df = df_city.filter(like='transportation').dropna(axis=0,how='all')
transportation_df.name = 'transportation_df'
collection_cost_df = df_city.filter(like='collection_cost').dropna(axis=0,how='all')
collection_cost_df.name = 'collection_cost_df'
collection_cover_df = df_city.filter(like='collection_cover').dropna(axis=0,how='all')
collection_cover_df.name = 'collection_cover_df'
management_df = df_city.filter(like='management_cost').dropna(axis=0,how='all')
management_df.name = 'management_df'
disposal_df = df_city.filter(like='disposal').dropna(axis=0,how='all')
disposal_df.name = 'disposal_df'
treatment_df = df_city.filter(like='treatment').dropna(axis=0,how='all')
treatment_df.name = 'treatment_df'
energy_df = df_city.filter(like='energy').dropna(axis=0,how='all')
energy_df.name = 'energy_df'
framework_df = df_city.filter(like='framework').dropna(axis=0,how='all')
framework_df.name = 'framework_df'


In [ ]:
def rectify_data(input_df):

    if len(input_df.select_dtypes(exclude=['object']).columns.tolist()):
        print("Cleaning numerical data")
        if input_df.name == 'informal_impact_df':
            for index in input_df['informal_sector_total_waste_pickers_number'].where(input_df['informal_sector_total_waste_pickers_number']>0):
                print(input_df['informal_sector_child_waste_pickers_number'])
                input_df.iloc[index]['informal_sector_child_waste_pickers_percent'] = 100*(input_df.iloc[index]['informal_sector_child_waste_pickers_number']/input_df.iloc[index]['informal_sector_total_waste_pickers_number'])
                input_df.iloc[index]['informal_sector_female_waste_pickers_percent'] = 100*(input_df.iloc[index]['informal_sector_female_waste_pickers_number']/input_df.iloc[index]['informal_sector_total_waste_pickers_number'])
                input_df = input_df.drop(columns=['informal_sector_child_waste_pickers_number', 'informal_sector_female_waste_pickers_number'])    

    for column in input_df.select_dtypes(include=['object']).columns.tolist():
        print("Cleaning categorical data", column)
        if input_df[column].str.isnumeric().any():
            print("Column has numbers in string")
            input_df[column] = pd.to_numeric(input_df[column], errors='coerce')
        else:
            print("Column has string data")
            value_list = ['yes', 'no ', 'door', 'curbside', 'centralized-drop-off', 'mulitple', 'other', 'flat fee per bussiness']
            for value in value_list:
                input_df.loc[input_df[column].str.contains(value, na=False, case=False), column] = value
    print("Data rectified")
    return input_df

def fill_num_data(input_df, levels, num_cols):
    
    for column in num_cols:
        print("Fill missing numerical data column ",column, "with mean of ", levels[0])
        input_df[column] = input_df[column].fillna(input_df.groupby(levels[0])[column].transform('mean'))
        if input_df[column].isnull().any():
            print("Fill missing numerical data column ",column, "with mean of ", levels[1])
            input_df[column] = input_df[column].fillna(input_df.groupby(levels[1])[column].transform('mean'))
        if input_df[column].isnull().any():
            print("Fill missing numerical data column ",column, "with mean of country_name")
            input_df[column] = input_df[column].fillna(input_df.groupby('country_name')[column].transform('mean'))
        if input_df[column].isnull().any():
            print("cannot fill missing numerical data. Drop row")
            input_df = input_df[input_df[column].notnull()]
    return input_df
def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
            
    return df
    



In [ ]:
def get_reportedData_distribution(input_df):
    reported_count = {column:input_df[column].dropna(how='all').shape[0] for column in  input_df.columns}
    plt.bar(range(len(reported_count)), list(reported_count.values()), tick_label=list(reported_count.keys()))
    lgd = plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.xticks(rotation=90)
    plt.ylabel('country_count')
    plt.savefig(input_df.name+'reportedData_vs_countryCount.png', format='png', bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.show()
    print("*********END OF REPORTED DATA***********")
    
    
def get_stacked_distribution(input_df, levels,cluster_levels):

    input_df2 = input_df.reset_index().groupby(levels).agg(    
    count_country =('country_name', "count"),# Get count of country for each group
    mean_population =(cluster_levels[1], 'mean'),# Get mean of the population for each group
    mean_msw =(cluster_levels[0], 'mean')# Get mean of the msw for each group
    )    
    
    print('mean population')
    input_df2['mean_population'].unstack().plot(kind='bar', stacked=False)#
    lgd = plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.xticks(rotation=90)
    plt.ylabel('mean population')
    plt.savefig(input_df.name+'_'+levels[0]+'_'+levels[1]+'_vs_meanPopulation.png', format='png', bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.show()
    print('mean msw')
    input_df2['mean_msw'].unstack().plot(kind='bar', stacked=False)#
    lgd = plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.xticks(rotation=90)
    plt.ylabel('mean msw')
    plt.savefig(input_df.name+'_'+levels[0]+'_'+levels[1]+'_vs_meanMSW.png', format='png', bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.show()
    #print('count_country', input_df2['count_country'].unstack())
    input_df2['count_country'].unstack().plot(kind='bar', stacked=True)#
    lgd = plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.ylabel('country_count')
    plt.xticks(rotation=90)
    plt.savefig(input_df.name+'_'+levels[0]+'_'+levels[1]+'_vs_countryCount.png', format='png', bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.show()
    num_cols = input_df.select_dtypes(exclude=['object']).columns.tolist()
    if num_cols:
        print("numerical data present. Plot mean and then get clusters")
        
        for level in levels:
            input_df3 = input_df.groupby(level).agg(['mean'])
            #print(input_df3)
            input_df3.plot(kind='bar', stacked=False)#
            lgd = plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
            plt.xticks(rotation=90)
            plt.ylabel('all_numerical_columns_mean')
            plt.savefig(input_df.name+'_'+level+'_'+'_all_numerical_columns_mean.png', format='png', bbox_extra_artists=(lgd,), bbox_inches='tight')
            #plt.show()
            
    else:
        print("For categorical data. Cannot get mean. Get clusters of ")
        
    #for column in input_df.select_dtypes(exclude=['object']).columns.tolist():#input_df.columns:
        #print(column)
        #input_df3[column].unstack().plot(kind='bar', stacked=False)#
        #plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        #plt.xticks(rotation=90)
        #plt.show()
    
    
    #input_df.plot.line(rot=90)
    print("*********END OF STACKED DISTRIBUTION***********")    
    

def plot_grouped_df(grouped_df, ax,  x, y, cmap = plt.cm.coolwarm):

    colors = cmap(np.linspace(0.5, 1, len(grouped_df)))
    c=np.array([colors])
    for i, (name,group) in enumerate(grouped_df):
        group.plot(ax=ax,
                   kind='scatter', 
                   x=x, y=y,
                   color=colors[i],
                   label = name)
def get_clusters(input_df, levels,cluster_level):
    
    plt_col = 1
    num_cols = input_df.select_dtypes(exclude=['object']).columns.tolist()
    plt_row_num = len(num_cols)#len(levels)* 
    cat_cols = input_df.select_dtypes(include=['object']).columns.tolist()
    plt_row_cat = len(cat_cols)
    
    count = 1
    for column in num_cols:       
        for level in levels:
            print("Numerical Data: ",column, level)

            fig, ax = plt.subplots(figsize=(10,2))        
            plot_grouped_df(input_df.reset_index().groupby(level),ax, x=column, y=cluster_level)
            #plt.subplot(plt_row,1,count)
            lgd = plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
            count = count +1
            plt.savefig(input_df.name+'_'+level+'_'+column+'.png', format='png', bbox_extra_artists=(lgd,), bbox_inches='tight')   
            #plt.show()
             
    count = 1
    for column in cat_cols:        
        for level in levels:
            print("Categorical Data:", column, level, cluster_level)
            sns.catplot(x=level, y=cluster_level, hue=column, kind="boxen", data=input_df.reset_index())
            #input_df.reset_index().groupby(level)[column].plot.bar()
            lgd = plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
            count = count +1
            plt.savefig(input_df.name+'_'+level+'_'+column+'.png', format='png', bbox_extra_artists=(lgd,), bbox_inches='tight')
            #plt.show()
    print("*********END OF CLUSTERS***********")    


    return 1
            

In [2]:
def check_relation(df):

    #Only use quant variables and drop any rows with missing values    
    num_cols = df.select_dtypes(exclude=['object']).columns.tolist()
    plt_row_num = len(num_cols)#len(levels)* 
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    plt_row_cat = len(cat_cols)
    input_dummy_df = df
    #print(input_dummy_df)
    if plt_row_num:
        input_dummy_df = fill_num_data(df, levels, num_cols)
    if plt_row_cat:
        input_dummy_df = create_dummy_df(input_dummy_df, cat_cols, dummy_na=True)
    

    input_df = input_dummy_df.reset_index()#   
    input_df = input_df[input_df['total_msw_total_msw_generated_tons_year'].notna()]
    input_df = input_df.set_index(predict_columns_name_indices)

    #Split into explanatory and response variables
    y = input_df['total_msw_total_msw_generated_tons_year']
    X = input_df.drop(['total_msw_total_msw_generated_tons_year'], axis=1)
    
    print("Check if x or y is nan or inf")
    print(np.any(np.isinf(X)))
    print(np.any(np.isinf(y)))
    print(np.any(np.isnan(X)))
    print(np.any(np.isnan(y)))
    print(X.columns[X.isnull().any()].tolist(), X.isnull().sum().sum())
    print(y.isnull().any(), y.isnull().sum().sum())
    #print(X[X.columns[X.isnull().any()].tolist()])
    print(pd.isnull(y).any().nonzero()[0])#np.where(y.isna())#y[y.isnull().any()])
    #print("CHeck for categorical data", input_dummy_df.select_dtypes(include=['object']).columns.tolist())
    #Split into train and test
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .50, random_state=42) 
    print(X_train.size, X_test.size)
    lm_model = LinearRegression(normalize=False) # Instantiate
    lm_model.fit(X_train, y_train) #Fit
        
    #Predict and score the model
    y_test_preds = lm_model.predict(X_test) 
    print("The r-squared score for your model was {} on {} values.".format(r2_score(y_test, y_test_preds), len(y_test)))

In [8]:
levels = ['income_id', 'region_id']#,
cluster_levels = ['total_msw_total_msw_generated_tons_year', 'population_population_number_of_people']
analysis_topic_dfList = [#informal_impact_df, 
                         #framework_df,
                         composition_df, 
                         separation_df, 
                         transportation_df, 
                         collection_cost_df,
                         collection_cover_df,
                         management_df, 
                         disposal_df, 
                         treatment_df, 
                         energy_df 
                        ]

for df in analysis_topic_dfList:
    #print(df)
    df = rectify_data(df)
    #print(df)
    get_reportedData_distribution(df)
    get_stacked_distribution(df, levels, cluster_levels)
    get_clusters(df, levels,cluster_levels[0])
    check_relation(df)

Cleaning numerical data
iso3c  region_id  country_name  income_id  city_name   total_msw_total_msw_generated_tons_year  population_population_number_of_people
AFG    SAS        Afghanistan   LIC        Jalalabad   58914.45                                 326585.0                                   NaN
                                           Kabul       1989250.00                               3700000.0                                  NaN
                                           HiratÂ      91644.70                                 337000.0                                   NaN
ARM    ECS        Armenia       UMC        Vanadzor    23596.97                                 86199.0                                    NaN
BGD    SAS        Bangladesh    LMC        Patuakhali  5220.00                                  65000.0                                    NaN
                                                                                                                              

TypeError: Cannot index by location index with a non-integer key